In [ ]:
img = [10,20,[10,20,255]]
img_ = img[:,:,::-1]
img_

TypeError: ignored

In [ ]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import cv2

def get_test_input():
  img = cv2.imread("dog-cycle-car.png")
  img = cv2.resize(img, (416,416))
  img_ = img[:,:,::-1].transpose((2,0,1)) 
  #img是【h,w,channel】，这里的img[:,:,::-1]是将第三个维度channel从opencv的BGR转化为pytorch的RGB，
  # 然后transpose((2,0,1))的意思是将[height,width,channel]->[channel,height,width]
  # ::-1 means "take everything in this dimension but backwards
  img_ = img_[np.newaxis,:,:,:]/255  #Add a channel at 0 (for batch) |then Normalise
  img_ = torch.from_numpy(img_).float()
  img_ = Variable(img_)
  return img_

處理圖片注意事項：
1. 用甚麼讀的，讀進來是h,w,c還是c,h,w(pytorch用)
2. channel裡面是RGB還是BGR
3. 新增sample數量/batch的維度
4. normalize(divided by 255)
5. 轉成pytorch tensor


In [ ]:
def parse_cfg(cfgfile):
    """
    输入: 配置文件路径
    返回值: 列表对象,其中每一个元素为一个字典类型对应于一个要建立的神经网络模块（层）
    
    """
    # 加载文件并过滤掉文本中多余内容
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')                        # store the lines in a list等价于readlines
    lines = [x for x in lines if len(x) > 0]               # 去掉空行
    lines = [x for x in lines if x[0] != '#']              # 去掉以#开头的注释行
    lines = [x.rstrip().lstrip() for x in lines]           # 去掉左右两边的空格(rstricp是去掉右边的空格，lstrip是去掉左边的空格)
    # cfg文件中的每个块用[]括起来最后组成一个列表，一个block存储一个块的内容，即每个层用一个字典block存储。
    block = {}
    blocks = []
    
    for line in lines:
        if line[0] == "[":               # 这是cfg文件中一个层(块)的开始           
            if len(block) != 0:          # 如果块内已经存了信息, 说明是上一个块的信息还没有保存
                blocks.append(block)     # 那么这个块（字典）加入到blocks列表中去
                block = {}               # 覆盖掉已存储的block,新建一个空白块存储描述下一个块的信息(block是字典)
            block["type"] = line[1:-1].rstrip()  # 把cfg的[]中的块名作为键type的值   
        else:
            key,value = line.split("=") #按等号分割
            block[key.rstrip()] = value.lstrip()#左边是key(去掉右空格)，右边是value(去掉左空格)，形成一个block字典的键值对
    blocks.append(block) # 退出循环，将最后一个未加入的block加进去
    # print('\n\n'.join([repr(x) for x in blocks]))
    return blocks

# 配置文件定义了6种不同type
# 'net': 相当于超参数,网络全局配置的相关参数
# {'convolutional', 'net', 'route', 'shortcut', 'upsample', 'yolo'}

# cfg = parse_cfg("cfg/yolov3.cfg")
# print(cfg)

In [ ]:
class EmptyLayer(nn.Module):
    """
    为shortcut layer / route layer 准备, 具体功能不在此实现，在Darknet类的forward函数中有体现
    """
    def __init__(self):
        super(EmptyLayer, self).__init__()
        

class DetectionLayer(nn.Module):
    '''yolo 检测层的具体实现, 在特征图上使用锚点预测目标区域和类别, 功能函数在predict_transform中'''
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors



def create_modules(blocks):
    net_info = blocks[0]     # blocks[0]存储了cfg中[net]的信息，它是一个字典，获取网络输入和预处理相关信息    
    module_list = nn.ModuleList() # module_list用于存储每个block,每个block对应cfg文件中一个块，类似[convolutional]里面就对应一个卷积块
    prev_filters = 3   #初始值对应于输入数据3通道，用来存储我们需要持续追踪被应用卷积层的卷积核数量（上一层的卷积核数量（或特征图深度））
    output_filters = []   #我们不仅需要追踪前一层的卷积核数量，还需要追踪之前每个层。随着不断地迭代，我们将每个模块的输出卷积核数量添加到 output_filters 列表上。
    
    for index, x in enumerate(blocks[1:]): #这里，我们迭代block[1:] 而不是blocks，因为blocks的第一个元素是一个net块，它不属于前向传播。
        module = nn.Sequential()# 这里每个块用nn.sequential()创建为了一个module,一个module有多个层
    
        #check the type of block
        #create a new module for the block
        #append to module_list
        
        if (x["type"] == "convolutional"):
            ''' 1. 卷积层 '''
            # 获取激活函数/批归一化/卷积层参数（通过字典的键获取值）
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False#卷积层后接BN就不需要bias
            except:
                batch_normalize = 0
                bias = True #卷积层后无BN层就需要bias
        
            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])
        
            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0
        
            # 开始创建并添加相应层
            # Add the convolutional layer
            # nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True)
            conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias = bias)
            module.add_module("conv_{0}".format(index), conv)
        
            #Add the Batch Norm Layer
            if batch_normalize:
                bn = nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)
        
            #Check the activation. 
            #It is either Linear or a Leaky ReLU for YOLO
            # 给定参数负轴系数0.1
            if activation == "leaky":
                activn = nn.LeakyReLU(0.1, inplace = True)
                module.add_module("leaky_{0}".format(index), activn)
                   
        elif (x["type"] == "upsample"):
            '''
            2. upsampling layer
            没有使用 Bilinear2dUpsampling
            实际使用的为最近邻插值
            '''
            stride = int(x["stride"])#这个stride在cfg中就是2，所以下面的scale_factor写2或者stride是等价的
            upsample = nn.Upsample(scale_factor = 2, mode = "nearest")
            module.add_module("upsample_{}".format(index), upsample)
                
        # route layer -> Empty layer
        # route层的作用：当layer取值为正时，输出这个正数对应的层的特征，如果layer取值为负数，输出route层向后退layer层对应层的特征
        elif (x["type"] == "route"):
            x["layers"] = x["layers"].split(',')
            #Start  of a route
            start = int(x["layers"][0])
            #end, if there exists one.
            try:
                end = int(x["layers"][1])
            except:
                end = 0
            #Positive anotation: 正值
            if start > 0: 
                start = start - index            
            if end > 0:# 若end>0，由于end= end - index，再执行index + end输出的还是第end层的特征
                end = end - index
            route = EmptyLayer()
            module.add_module("route_{0}".format(index), route)
            if end < 0: #若end<0，则end还是end，输出index+end(而end<0)故index向后退end层的特征。
                filters = output_filters[index + start] + output_filters[index + end]
            else: #如果没有第二个参数，end=0，则对应下面的公式，此时若start>0，由于start = start - index，再执行index + start输出的还是第start层的特征;若start<0，则start还是start，输出index+start(而start<0)故index向后退start层的特征。
                filters= output_filters[index + start]
    
        #shortcut corresponds to skip connection
        elif x["type"] == "shortcut":
            shortcut = EmptyLayer() #使用空的层，因为它还要执行一个非常简单的操作（加）。没必要更新 filters 变量,因为它只是将前一层的特征图添加到后面的层上而已。
            module.add_module("shortcut_{}".format(index), shortcut)
            
        #Yolo is the detection layer
        elif x["type"] == "yolo":
            mask = x["mask"].split(",")
            mask = [int(x) for x in mask]
    
            anchors = x["anchors"].split(",")
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in mask]
    
            detection = DetectionLayer(anchors)# 锚点,检测,位置回归,分类，这个类见predict_transform中
            module.add_module("Detection_{}".format(index), detection)
                              
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        
    return (net_info, module_list)

筆記
1. #卷积层后无BN层就需要bias???
2. leakyReLU的參數、inplace?
3. over/undersampling,  up/downsampling
4. route裡面的filters為何要相加？

In [ ]:

class Darknet(nn.Module):
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile) #调用parse_cfg函数
        self.net_info, self.module_list = create_modules(self.blocks)#调用create_modules函数
        
    def forward(self, x, CUDA):
        modules = self.blocks[1:] # 除了net块之外的所有，forward这里用的是blocks列表中的各个block块字典
        outputs = {}   #We cache the outputs for the route layer，待會會放 "層數":"特徵"
        
        write = 0#write表示我们是否遇到第一个检测。write=0，则收集器尚未初始化，write=1，则收集器已经初始化，我们只需要将检测图与收集器级联起来即可。
        for i, module in enumerate(modules):      #iter cfg檔中的文字blocks  
            module_type = (module["type"])
            
            if module_type == "convolutional" or module_type == "upsample":
                x = self.module_list[i](x) #傳遞特徵到層裡面
    
            elif module_type == "route":
                layers = module["layers"]
                layers = [int(a) for a in layers]
    
                if (layers[0]) > 0:
                    layers[0] = layers[0] - i
                # 如果只有一层时。从前面的if (layers[0]) > 0:语句中可知，如果layer[0]>0，则输出的就是当前layer[0]这一层的特征,如果layer[0]<0，输出就是从route层(第i层)向后退layer[0]层那一层得到的特征 
                if len(layers) == 1:
                    x = outputs[i + (layers[0])]
                #第二个元素同理 
                else:
                    if (layers[1]) > 0:
                        layers[1] = layers[1] - i #若layer[1] = 61，代表要跳到第61層(第61個block) ，則先減掉現在在的層數，再於下方加總
    
                    map1 = outputs[i + layers[0]]
                    map2 = outputs[i + layers[1]]
                    x = torch.cat((map1, map2), 1)#第二个参数设为 1,这是因为我们希望将特征图沿anchor数量的维度级联起来。
                
    
            elif  module_type == "shortcut":
                from_ = int(module["from"])
                x = outputs[i-1] + outputs[i+from_] # 求和运算，它只是将前一层的特征图添加到后面的层上而已
            
            elif module_type == 'yolo':        
                anchors = self.module_list[i][0].anchors
                #从net_info(实际就是blocks[0]，即[net])中get the input dimensions
                inp_dim = int (self.net_info["height"])
        
                #Get the number of classes
                num_classes = int (module["classes"])
        
                #Transform 
                x = x.data # 这里得到的是预测的yolo层feature map
                # 在util.py中的predict_transform()函数利用x(是传入yolo层的feature map)，得到每个格子所对应的anchor最终得到的目标
                # 坐标与宽高，以及出现目标的得分与每种类别的得分。经过predict_transform变换后的x的维度是(batch_size, grid_size*grid_size*num_anchors, 5+类别数量)
                x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                 
                if not write:              #if no collector has been intialised. 因为一个空的tensor无法与一个有数据的tensor进行concatenate操作，
                    detections = x #所以detections的初始化在有预测值出来时才进行，
                    write = 1   #用write = 1标记，当后面的分数出来后，直接concatenate操作即可。
        
                else:  
                    '''
                    变换后x的维度是(batch_size, grid_size*grid_size*num_anchors, 5+类别数量)，这里是在维度1上进行concatenate，即按照
                    anchor数量的维度进行连接，对应教程part3中的Bounding Box attributes图的行进行连接。yolov3中有3个yolo层，所以
                    对于每个yolo层的输出先用predict_transform()变成每行为一个anchor对应的预测值的形式(不看batch_size这个维度，x剩下的
                    维度可以看成一个二维tensor)，这样3个yolo层的预测值按照每个方框对应的行的维度进行连接。得到了这张图处所有anchor的预测值，后面的NMS等操作可以一次完成
                    '''
                    detections = torch.cat((detections, x), 1)# 将在3个不同level的feature map上检测结果存储在 detections 里
        
            outputs[i] = x

            
        def load_weights(self, weightfile):
          fp = open(weightfile, "rb") #r :　read as str/ rb: read as bytes
          #The first 5 values are header information   
          # 1. Major version number 
          # 2. Minor Version Number
          # 3. Subversion number 
          # 4,  5. Images seen by the network (during training)
          header = np.fromfile(fp, dtype = np.int32, count=5)
          self.header = torch.from_numpy(header)
          self.seen = self.header[3]

          weights = np.fromfile(fp, dtype = np.float32)

          ptr = 0
          for i in range(len(self.module_list)):
            module_type = self.blocks[i+1]["type"] #blocks[0]是net描述文字區，真正的module文字要從blocks[1]開始
              #If module_type is convolutional load weights
              #Otherwise ignore.

            if module_type == "convolutional":
              model = self.module_list[i]
              try:
                batch_normalize = int(self.blocks[i+1]["batch_normalize"]) 
                #有就 = 1
              except:
                batch_normalize = 0

              conv = model[0]

              if(batch_normalize):
                bn = model[1] #若有bn層，即在conv下一層
                num_bn_bias = bn.bias.numel() #numel:獲得矩陣元素個數。此處為獲得預設的權重參數個數
                
                #Load the weights
                bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_bias])
                ptr += num_bn_biases

                bn_weights = torch.from_numpy(weights[ptr:ptr + num_bn_bias])
                ptr += num_bn_biases

                bn_running_mean = torch.from_numpy(weights[ptr:ptr + num_bn_bias])
                ptr += num_bn_biases

                bn_running_var = torch.from_numpy(weights[ptr:ptr + num_bn_bias])
                ptr += num_bn_biases


                #Cast the loaded weights into dims of model weights. 
                bn_biases = bn_biases.view_as(bn.bias.data)
                bn_weights = bn_weights.view_as(bn.weight.data)
                bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                bn_running_var = bn_running_var.view_as(bn.running_var)

                #Copy the data to model 将从weights文件中得到的权重bn_biases复制到model中(bn.bias.data)
                bn.bias.data.copy_(bn_biases)
                bn.weights.data.copy_(bn_weights)
                bn.running_mean.copy_(bn_running_mean)
                bn.running_var.copy_(bn_running_var)
              
              else:
                #如果 batch_normalize 的检查结果不是 True，只需要加载卷积层的偏置项
                #Number of biases 
                num_biases = conv.bias.numel()

                conv_biases = torch.from_numpy(weights[ptr:ptr + num_biases])
                ptr = ptr + num_biases

                conv_biases = conv_biases.view_as(conv.bias.data)

                conv.bias.data.copy_(conv_biases)
              
              # 有bn:加載bn的bias、weights、running_mean & var
              # 沒bn:加載conv的bias
              # 然後都要加載conv的weights

              num_weights = conv.weights.numel()

              conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
              ptr+=num_weights

              conv_weights = conv_weights.view_as(conv.weights.data)
              conv.weights.data.copy_(conv_weights)          










Darknet:C語言的開源框架、用已記錄下來cfg檔中的參數來run
1. 用迴圈遍歷每個BLOCK，跑每個BLOCK的Layer並傳遞特徵x，然後放在outputs中(outputs：{1:x1, 2:x2, 3:x3})
2. layers 的記法為 num1, num2(optional)，其中若為負數，則表示往回幾個blocks，若為正數，表示跳到指定第幾個block
3. shortcut 為何要相加(ResNet的運用?)将输入的特征图，与输出特征图对应维度进行相加，即y = f(x) + x
4. route 為何要concat: concat操作源于DenseNet网络的设计思路，将特征图按照通道维度直接进行拼接，例如8*8*16的特征图与8*8*16的特征图拼接后生成8*8*32的特征图。

Weights
1. 獲得原本模型預設的conv/bn參數數量k
2. 用k去取得weights檔裡對應的數字並轉成torch(from_numpy:numpy->torch)
3. 將獲得的weights數字們轉成模型需要的矩陣形狀(view_as)
4. 將其copy到模型的data裡
5. weights檔長怎樣?
6. bn.bias.data長怎樣

util.py。这个程序包含了predict_transform函数（Darknet类中的forward函数要用到），write_results函数使我们的输出满足 objectness 分数阈值和非极大值抑制（NMS），以得到「真实」检测结果。还有prep_image和letterbox_image等图片预处理函数等（前者用来将numpy数组转换成PyTorch需要的的输入格式，后者用来将图片按照纵横比进行缩放，将空白部分用(128,128,128)填充）。话不多说，直接看util.py的注释。

In [ ]:
def unique(tensor):#因为同一类别可能会有多个真实检测结果，所以我们使用unique函数来去除重复的元素，即一类只留下一个元素，达到获取任意给定图像中存在的类别的目的。
  tensor_np = tensor.cpu().numpy()
  unique_np = np.unique(tensor_np)#np.unique该函数是去除数组中的重复数字，并进行排序之后输出
  unique_tensor = torch.from_numpy(unique_np)
  # 复制数据
  tensor_res = tensor.new(unique_tensor.shape)# new(args, *kwargs) 构建[相同数据类型]的新Tensor
  tensor_res.copy_(unique_tensor)
  # why not tensor_res = unique_tensor 就好
  return tensor_res

def bbox_iou(box1, box2):
      """
    Returns the IoU of two bounding boxes 
    
    
    """
    #Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3] 
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3] 

    #get the corrdinates of the intersection rectangle
    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    #Intersection area
    # Intersection area 这里没有对inter_area为负的情况进行判断，后面计算出来的IOU就可能是负的
    # clamp:避免沒有交集時變成負值
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 +1, min=0) * torch.clamp(inter_rect_y2 - inter_rect_y1 +1 , min=0) 

    b1_area = (b1_x2 - b1_x1 +1 )*(b1_y2 - b1_y1 +1 )
    b2_area = (b2_x2 - b2_x1 +1 )*(b2_y2 - b2_y1 +1 )

    iou = inter_area/(b1_area + b2_area - inter_area)
    return iou

  def  predict_transform(prediction, inp_dim, anchors, num_classes, CUDA = True)
    """
    在特征图上进行多尺度预测, 在GRID每个位置都有三个不同尺度的锚点.predict_transform()利用一个scale得到的feature map预测得到的每个anchor的属性(x,y,w,h,s,s_cls1,s_cls2...),其中x,y,w,h
    是在网络输入图片坐标系下的值,s是方框含有目标的置信度得分，s_cls1,s_cls_2等是方框所含目标对应每类的概率。输入的feature map(prediction变量) 
    维度为(batch_size, num_anchors*bbox_attrs, grid_size, grid_size)，类似于一个batch彩色图片BxCxHxW存储方式。参数见predict_transform()里面的变量。
    并且将结果的维度变换成(batch_size, grid_size*grid_size*num_anchors, 5+类别数量)的tensor，同时得到每个方框在网络输入图片(416x416)坐标系下的(x,y,w,h)以及方框含有目标的得分以及每个类的得分。
    """
    batch_size = prediction.size(0)
    # stride表示的是整个网络的步长，等于图像原始尺寸与yolo层输入的feature mapr尺寸相除，因为输入图像是正方形，所以用高相除即可
    stride = inp_dim//prediction.size(2)#416//13= 32  假設feature_map是13x13

    # feature map每条边格子的数量，416//32 = 13
    grid_size = inp_dim//stride

    bbox_attrs = 5 + num_classes #框的屬性包含5(x,y,w,h,置信度=1or0) + 各類別機率

    num_anchors = len(anchors)

    # 输入的prediction维度为(batch_size, num_anchors * bbox_attrs, grid_size, grid_size)，类似于一个batch彩色图片BxCxHxW
    # 存储方式，将它的维度变换成(batch_size, bbox_attrs*num_anchors, grid_size*grid_size)
    prediction = prediction.view(batch_size, bbox_attrs*num_anchors, grid_size*grid_size) #300x (5x85) x (16x16)

    #contiguous：view只能用在contiguous的variable上。如果在view之前用了transpose, permute等，需要用contiguous()来返回一个contiguous copy。
    prediction = prediction.transpose(1,2).contiguous()

    # 将prediction维度转换成(batch_size, grid_size*grid_size*num_anchors, bbox_attrs)。不看batch_size，
    # (grid_size*grid_size*num_anchors, bbox_attrs)相当于将所有anchor按行排列，即一行对应一个anchor属性，此时的属性仍然是feature map得到的值   
    prediction = prediction.view(batch_size, grid_size*grid_size*num_anchors, bbox_attrs ) 
    
    # 锚点的维度与net块的height和width属性一致。这些属性描述了输入图像的维度，比feature map的规模大（二者之商即是步幅）。因此，我们必须使用stride分割锚点。变换后的anchors是相对于最终的feature map的尺寸
    anchors = [(a[0]/stride, a[1]/stride) for a in anchors]

    #Sigmoid the tX, tY. and object confidencce.tx与ty为预测的坐标偏移值
    prediction[:,:,0] = torch.sigmoid(prediction[:,:,0])
    prediction[:,:,1] = torch.sigmoid(prediction[:,:,1])
    prediction[:,:,4] = torch.sigmoid(prediction[:,:,4])

    #这里生成了每个格子的左上角坐标，生成的坐标为grid x grid的二维数组，a，b分别对应这个二维矩阵的x,y坐标的数组，a,b的维度与grid维度一样。每个grid cell的尺寸均为1，故grid范围是[0,12]（假如当前的特征图13*13）
    grid = np.arrange(grid_size)
    a,b = np.meshgrid(grid, grid) #(0,0 0,1...0,12, 1,0...1,12 ... 12,0...12,12)
    x_offset = torch.FloatTensor(a).view(-1,1)
    y_offset = torch.FloarTensor(b).view(-1,1)

    if CUDA:
      x_offset = x_offset.cuda()
      y_offset = y_offset.cuda()
    
    #这里的x_y_offset对应的是最终的feature map中每个格子的左上角坐标，比如有13个格子，刚x_y_offset的坐标就对应为(0,0),(0,1)…(12,12) .view(-1, 2)将tensor变成两列，unsqueeze(0)在0维上添加了一维。
    x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1, num_anchors).view(-1,2).unsqueeze()
    prediction[:,:,:2] += x_y_offset#bx=sigmoid(tx)+cx,by=sigmoid(ty)+cy
    
    if CUDA:
      anchors = anchors.cuda()

    # 这里的anchors本来是一个长度为6的list(三个anchors每个2个坐标)，然后在0维上(行)进行了grid_size*grid_size个复制，在1维(列)上
    # 一次复制(没有变化)，即对每个格子都得到三个anchor。Unsqueeze(0)的作用是在数组上添加一维，这里是在第0维上添加的。添加grid_size是为了之后的公式bw=pw×e^tw的tw。    
    anchors = anchors.repeat(grid_size*grid_size, 1).unsqueeze(0)

    #对网络预测得到的矩形框的宽高的偏差值进行指数计算，然后乘以anchors里面对应的宽高(这里的anchors里面的宽高是对应最终的feature map尺寸grid_size)，
    # 得到目标的方框的宽高，这里得到的宽高是相对于在feature map的尺寸
    prediction[:,:,2:4] = torch.exp(prediction[:,:,2:4])*anchors #[(pw,ph),(pw,ph)...] * [(exp(tw),exp(th)),(exp(tw),exp(th))...] (對應相乘)
    
    # 这里得到每个anchor中每个类别的得分。将网络预测的每个得分用sigmoid()函数计算得到
    prediction[:,:,5: 5+ num_classes] = torch.sigmoid((prediction[:,:, 5+ num_classes]))

    prediction[:,:,:4] *= stride
    #将相对于最终feature map的方框坐标和尺寸映射回输入网络图片(416x416)，即将方框的坐标乘以网络的stride即可

    return prediction

unique func
1. .cdu().numpy()
That’s because numpy doesn’t support CUDA, so there’s no way to make it use GPU memory without a copy to CPU first. Remember that .numpy() doesn’t do any copy, but returns an array that uses the same memory as the tensor.
2. clamp():限制輸出在min max間(小於min就輸出min，大於max就輸出max)
3. why IOU計算要+1
4. Contiguous跟記憶體存取有關(執行view記憶體需連續)
5. meshgrid 生成範圍內的每個網格樣本點(以code為例，即生成0,0 0,1...0,12, 1,0...1,12 ... 12,0...12,12)
6. 對f_map的每格都做計算，最後用置信度篩出最可能的
在prediction view的時候就把predictiont拆成batch_size x grid_size*grid_size*nums_anchors x bbox_attrs
中間那項即在每格都計算一個數值(長向量)

In [ ]:
import torch
anchors = torch.tensor([(13,30),(30,36),(36,40)])
grid_size = 13
anchors = anchors.repeat(grid_size*grid_size, 1).unsqueeze(0)
print(anchors[:,0:10,0:2])

tensor([[[13, 30],
         [30, 36],
         [36, 40],
         [13, 30],
         [30, 36],
         [36, 40],
         [13, 30],
         [30, 36],
         [36, 40],
         [13, 30]]])


In [ ]:
    '''
    必须使我们的输出满足 objectness 分数阈值和非极大值抑制（NMS），以得到后文所提到的「真实」检测结果。要做到这一点就要用 write_results函数。
    函数的输入为预测结果、置信度（objectness 分数阈值）、num_classes（我们这里是 80）和 nms_conf（NMS IoU 阈值）。
    write_results()首先将网络输出方框属性(x,y,w,h)转换为在网络输入图片(416x416)坐标系中，方框左上角与右下角坐标(x1,y1,x2,y2)，以方便NMS操作。
    然后将方框含有目标得分低于阈值的方框去掉，提取得分最高的那个类的得分max_conf，同时返回这个类对应的序号max_conf_score,
    然后进行NMS操作。最终每个方框的属性为(ind,x1,y1,x2,y2,s,s_cls,index_cls)，ind 是这个方框所属图片在这个batch中的序号，
    x1,y1是在网络输入图片(416x416)坐标系中，方框左上角的坐标；x2,y2是在网络输入图片(416x416)坐标系中，方框右下角的坐标。
    s是这个方框含有目标的得分,s_cls是这个方框中所含目标最有可能的类别的概率得分，index_cls是s_cls对应的这个类别所对应的序号.
    '''
def write_results(prediction, confidence, num_classes, nms_conf=0.4):
  # confidence: 输入的预测shape=(1,10647, 85)。conf_mask: shape=(1,10647) => 增加一维度之后 (1, 10647, 1) 
  #85類只抓一類出來
  conf_mask = (prediction[:,:,4]>confidence).float().unsqueeze(2)
  prediction = prediction*conf_mask
  #與True/False相乘 =>  
  # 小于置信度的条目值全为0, 剩下部分不变。conf_mask中含有目标的得分小于confidence的方框所对应的含有目标的得分为0，
  #根据numpy的广播原理，它会扩展成与prediction维度一样的tensor，所以含有目标的得分小于confidence的方框所有的属性都会变为0，故如果没有检测任何有效目标,则返回值为0

  '''
  保留预测结果中置信度大于阈值的bbox
  下面开始为nms准备
  '''

  # prediction的前五个数据分别表示 (Cx, Cy, w, h, score)，这里创建一个新的数组，大小与predicton的大小相同   
  box_corner = prediction.new(prediction.shape)
  '''
  我们可以将我们的框的 (中心 x, 中心 y, 高度, 宽度) 属性转换成 (左上角 x, 左上角 y, 右下角 x, 右下角 y)
  这样做用每个框的两个对角坐标能更轻松地计算两个框的 IoU
  '''
  box_corner[:,:,0] = (prediction[:,:,0] - prediction[:,:,2]/2)# x1 = Cx - w/2
  box_corner[:,:,1] = (prediction[:,:,1] - prediction[:,:,3]/2)# y1 = Cy - h/2
  box_corner[:,:,2] = (prediction[:,:,0] + prediction[:,:,2]/2)# x2 = Cx + w/2 
  box_corner[:,:,3] = (prediction[:,:,1] + prediction[:,:,3]/2)# y2 = Cy + h/2
  prediction[:,:,:4] = box_corner[:,:,:4]# 计算后的新坐标复制回去

  batch_size = prediction.size(0)
  # 第0个维度是batch_size
  # output = prediction.new(1, prediction.size(2)+1) # shape=(1,85+1)
  write = False# 拼接结果到output中最后返回
  #对每一张图片得分的预测值进行NMS操作，因为每张图片的目标数量不一样，所以有效得分的方框的数量不一样，没法将几张图片同时处理，因此一次只能完成一张图的置信度阈值的设置和NMS,不能将所涉及的操作向量化.
  #所以必须在预测的第一个维度上（batch数量）上遍历每张图片，将得分低于一定分数的去掉，对剩下的方框进行进行NMS

  for ind in range(batch_size):
    image_pred = prediction[ind]# 选择此batch中第ind个图像的预测结果,image_pred对应一张图片中所有方框的坐标(x1,y1,x2,y2)以及得分，是一个二维tensor 维度为10647x85

    







1. confidence是一個值?

In [4]:
10647/3/13/13

21.0